### 라이브러리

In [1]:
import pandas as pd
import os
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration
from gensim.summarization.summarizer import summarize
from newspaper import Article

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


### 데이터 전처리

In [27]:
news = pd.read_csv('news_crawing/data/저상버스_불편.csv', encoding='utf-8')
news

,기사제목,기사내용
0,대전·세종 BRT 도로에 2층 전기저상버스 달린다,[\r\n다음 달 14일부터 B1 노선에 2대 운행…하루 20회\r\n\r\n\r\...
1,"대전시, 2층 전기 저상버스 내달부터 운행",[\r\n장애인-교통약자 등 이용 가능\r\n\r\n\r\n\r\n대전시가 중부권 ...
2,[무등의 아침] 11일째 멈춰 선 목포 시내버스…보조금에만 목매는 회사,"[\r\n- ""목포 시내버스 회사 노조, 18일 파업 돌입..11일째 이어져""- ""..."
3,‘버스 운행 방해’ 박경석 전장연 대표 1심 집행유예,"[\r\n버스 앞문, 몸 쇠사슬로 연결…23분간 운행 방해1심 징역 4월·집행유예 ..."
4,"法 “버스·지하철 운행 방해, 정당화 안 돼” 전장연 대표 집행유예",[\r\n징역 4개월·집유 2년 선고“집회 방식 재고 당부했음에도…”박 대표 “선생...
...,...,...
1267,"""가짜로 살고 싶지 않아 '나쁜 장애인' 됐습니다"" [청년정치 와글와글]",[\r\n[청년정치와글와글-인터뷰] 변재원 전국장애인차별철폐연대 정책국장 [이진순]...
1268,"""장애인정책 확대하라"" 장애인단체 '도로점거'…청주도심 한때 정체",[\r\n장애인 관련 정책 확대·지사 면담 요구 3시간 점거이시종 지사 면담 약속 ...
1269,"낮은 문턱, 계단 사라진 놀이터... '무장애 도시' 아시나요",[\r\n자체 조례 제정 등 장애물 없는 생활환경 실현 나선 지역들 [월간 옥이네]...
1270,"'우당 이회영' 기념관 완공, 오세훈의 '남산 르네상스' 마무리",[\r\n1920년 독립군의 화력 뒷받침한 '체코 군단' 무기도 특별전시[손병관 기...


In [28]:
news = news.drop(columns=['기사제목'], axis=0)
news["기사내용"] = news["기사내용"].str.replace("\t", "").str.replace(".", "\n")
news

C:\Users\admin\AppData\Local\Temp\ipykernel_13936\2109014240.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  news["기사내용"] = news["기사내용"].str.replace("\t", "").str.replace(".", "\n")


,기사내용
0,[\r\n다음 달 14일부터 B1 노선에 2대 운행…하루 20회\r\n\r\n\r\...
1,[\r\n장애인-교통약자 등 이용 가능\r\n\r\n\r\n\r\n대전시가 중부권 ...
2,"[\r\n- ""목포 시내버스 회사 노조, 18일 파업 돌입\n\n11일째 이어져""-..."
3,"[\r\n버스 앞문, 몸 쇠사슬로 연결…23분간 운행 방해1심 징역 4월·집행유예 ..."
4,[\r\n징역 4개월·집유 2년 선고“집회 방식 재고 당부했음에도…”박 대표 “선생...
...,...
1267,[\r\n[청년정치와글와글-인터뷰] 변재원 전국장애인차별철폐연대 정책국장 [이진순]...
1268,[\r\n장애인 관련 정책 확대·지사 면담 요구 3시간 점거이시종 지사 면담 약속 ...
1269,[\r\n자체 조례 제정 등 장애물 없는 생활환경 실현 나선 지역들 [월간 옥이네]...
1270,[\r\n1920년 독립군의 화력 뒷받침한 '체코 군단' 무기도 특별전시[손병관 기...


In [29]:
# 요약 글 데이터프레임으로 만들어주기
for i in range(0, len(news)):
    try:
        text = news['기사내용'][i]
        # 문자열로 만들어주기
        text = (''.join(summarize(text, ratio=0.4).split('\n')))
        # print(text_list)
    except Exception as e:
        print(i,",", end='')
        continue

96 ,247 ,655 ,837 ,838 ,919 ,

In [30]:
news = news.drop([96 ,247 ,655 ,837 ,838 ,919 ,])
news = news.reset_index(drop=True)
news

,기사내용
0,[\r\n다음 달 14일부터 B1 노선에 2대 운행…하루 20회\r\n\r\n\r\...
1,[\r\n장애인-교통약자 등 이용 가능\r\n\r\n\r\n\r\n대전시가 중부권 ...
2,"[\r\n- ""목포 시내버스 회사 노조, 18일 파업 돌입\n\n11일째 이어져""-..."
3,"[\r\n버스 앞문, 몸 쇠사슬로 연결…23분간 운행 방해1심 징역 4월·집행유예 ..."
4,[\r\n징역 4개월·집유 2년 선고“집회 방식 재고 당부했음에도…”박 대표 “선생...
...,...
1261,[\r\n[청년정치와글와글-인터뷰] 변재원 전국장애인차별철폐연대 정책국장 [이진순]...
1262,[\r\n장애인 관련 정책 확대·지사 면담 요구 3시간 점거이시종 지사 면담 약속 ...
1263,[\r\n자체 조례 제정 등 장애물 없는 생활환경 실현 나선 지역들 [월간 옥이네]...
1264,[\r\n1920년 독립군의 화력 뒷받침한 '체코 군단' 무기도 특별전시[손병관 기...


In [31]:
# 리스트로 담기
sum_list = []

for i in range(0, len(news)):
    text = news['기사내용'][i]
    text_list = (''.join(summarize(text, ratio=0.4).split('\n')))
    
    sum_list.append(text_list)

In [32]:
total = pd.DataFrame(sum_list, columns=['기사요약'])
total

,기사요약
0,다음 달 14일부터 B1 노선에 2대 운행…하루 20회\r대전시가 도입하는 2층 전...
1,대전시가 중부권 최초로 다음 달 14일부터 운행하는 2층 전기 저상버스대전시 제공대...
2,"지난해에는 사측이 휴업 신청""- ""목포시, 재정지원금 늘리면서 시내버스 문제 매듭 ..."
3,박경석 장애인차별철폐연대 공동대표[연합][헤럴드경제=유동현 기자] 장애인 이동권 보...
4,박경석 전국장애인차별철폐연대(전장연) 대표가 18일 버스 운행방해 사건 1심 선고 ...
...,...
1261,[청년정치와글와글-인터뷰] 변재원 전국장애인차별철폐연대 정책국장 [이진순] ...
1262,충북장애인차별철폐연대 회원들이 13일 충북도청 서문 앞 도로를 점거하고 충북도의 장...
1263,자체 조례 제정 등 장애물 없는 생활환경 실현 나선 지역들 [월간 옥이네] \r...
1264,ⓒ 서울시 제공 조선총독부 관저와 중앙정보부 등이 자리잡았던 남산 예장자락이 시...


In [33]:
total.to_csv('res/news_summary/저상버스_불편_요약_0.4.csv')